In [36]:
from langchain.document_loaders import DirectoryLoader,TextLoader,PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from langchain.chains import RetrievalQA
import gradio as gr
import ollama 
import os
import glob
import shutil

In [37]:
!pip install -U langchain-ollama

In [38]:
!pip install pymupdf

In [40]:
!pip install -U langchain-ollama

In [39]:
db_name="vector_db"

In [51]:
import gradio as gr
import os
import shutil
from langchain.document_loaders import PyMuPDFLoader
from langchain_ollama import ChatOllama

conversation_chain=None
def process_pdf(pdf_file):
    global conversation_chain
    UPLOAD_FOLDER = "./data"
    if os.path.exists(UPLOAD_FOLDER):
        for filename in os.listdir(UPLOAD_FOLDER):
            filepath=os.path.join(UPLOAD_FOLDER,filename)
            if os.path.isfile(filepath) or os.path.islink(filepath):
                os.remove(filepath)
    else:
        os.mkdir(UPLOAD_FOLDER)

    
    shutil.copy(pdf_file.name, UPLOAD_FOLDER)
    saved_path = os.path.join(UPLOAD_FOLDER, os.path.basename(pdf_file.name))
    # print(saved_path,os.path.basename(pdf_file.name))
    loader = PyMuPDFLoader(saved_path)
    documents = loader.load()  
    text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_documents(documents)
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    if os.path.exists(db_name):
        Chroma(persist_directory=db_name,embedding_function=embeddings).delete_collection()
    vectorstore=Chroma.from_documents(documents=chunks,embedding=embeddings,persist_directory=db_name)
    llm=ChatOllama(model="llama3.2",temperature=0.7)
    memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
    retriever=vectorstore.as_retriever()
    conversation_chain=ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever,memory=memory)
    return "✅ PDF processed and QA system ready! You can now ask questions."

    




In [52]:
def chat(message,history):
    global conversation_chain
    result=conversation_chain.invoke({"question":message})
    return result["answer"]
with gr.Blocks() as demo:
    gr.Markdown("## 📄 Upload PDF & Ask Questions")

    pdf_input = gr.File(label="Upload PDF", file_types=[".pdf"])
    upload_status = gr.Textbox(label="Status", interactive=False)

    question_input = gr.Textbox(label="Your Question")
    answer_output = gr.Textbox(label="Answer", interactive=False, lines=5)

    ask_button = gr.Button("Ask")

    pdf_input.change(fn=process_pdf, inputs=pdf_input, outputs=upload_status)
    ask_button.click(fn=chat, inputs=question_input, outputs=answer_output)

demo.launch(inbrowser=True)


c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1023: UserWarning: Expected 2 arguments for function <function chat at 0x000001680DFF8180>, received 1.
  warnings.warn(
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py:1027: UserWarning: Expected at least 2 arguments for function <function chat at 0x000001680DFF8180>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7882
* To create a public link, set `share=True` in `launch()`.


c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
c:\Users\surya\anaconda3\envs\llms\Lib\site-packages\gradio\helpers.py:977: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
